In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from collections import Counter
from surprise import Dataset, Reader
from surprise import accuracy
from surprise.model_selection import cross_validate, train_test_split
from surprise.prediction_algorithms import SVD
from surprise.prediction_algorithms import KNNWithMeans, KNNBasic, KNNBaseline
from surprise.model_selection import GridSearchCV

In [19]:
df_0 = pd.read_csv('../../../data/fps_trimmed_00.csv', error_bad_lines=False, engine="python")
df_0

Skipping line 10436: field larger than field limit (131072)
Skipping line 832659: field larger than field limit (131072)
Skipping line 1019250: field larger than field limit (131072)
Skipping line 1192361: field larger than field limit (131072)
Skipping line 1317332: unexpected end of data


,steamid,appid,app_title,app_tags,review,voted_up,fps
0,76561198107294407,10,"<div class=""apphub_AppName"" id=""appHubAppName""...","['Action', 'FPS', 'Multiplayer', 'Shooter', 'C...",best gameplay ever created,True,True
1,76561198011733201,10,"<div class=""apphub_AppName"" id=""appHubAppName""...","['Action', 'FPS', 'Multiplayer', 'Shooter', 'C...",Timeless. Thank you volvo.,True,True
2,76561198168961276,10,"<div class=""apphub_AppName"" id=""appHubAppName""...","['Action', 'FPS', 'Multiplayer', 'Shooter', 'C...",this game literally made me love guns and fps ...,True,True
3,76561198957877160,10,"<div class=""apphub_AppName"" id=""appHubAppName""...","['Action', 'FPS', 'Multiplayer', 'Shooter', 'C...",If this gets a rework in 2021 it's going to be...,True,True
4,76561199050314447,10,"<div class=""apphub_AppName"" id=""appHubAppName""...","['Action', 'FPS', 'Multiplayer', 'Shooter', 'C...",ok,True,True
...,...,...,...,...,...,...,...
1317321,76561198127368408,730,"<div class=""apphub_AppName"" id=""appHubAppName""...","['FPS', 'Shooter', 'Multiplayer', 'Competitive...",Just Started to play this game and already lik...,True,True
1317322,76561198127721958,730,"<div class=""apphub_AppName"" id=""appHubAppName""...","['FPS', 'Shooter', 'Multiplayer', 'Competitive...",I like pah-tahtoes.,True,True
1317323,76561198073650871,730,"<div class=""apphub_AppName"" id=""appHubAppName""...","['FPS', 'Shooter', 'Multiplayer', 'Competitive...",an amazing game but it needs some fix,True,True
1317324,76561198127503594,730,"<div class=""apphub_AppName"" id=""appHubAppName""...","['FPS', 'Shooter', 'Multiplayer', 'Competitive...","Buy it, u wont regret it trust me",True,True


In [7]:
df_1 = pd.read_csv('../../../data/fps_trimmed_01.csv', error_bad_lines=False, engine="python")
df_1

,steamid,appid,app_title,app_tags,review,voted_up,fps
0,76561198253393713,4000,"<div class=""apphub_AppName"" id=""appHubAppName""...","['Sandbox', 'Multiplayer', 'Funny', 'Moddable'...",I mean its alright I guess,True,True
1,76561199150336388,4000,"<div class=""apphub_AppName"" id=""appHubAppName""...","['Sandbox', 'Multiplayer', 'Funny', 'Moddable'...",i love gerrys mod,True,True
2,76561199161506056,4000,"<div class=""apphub_AppName"" id=""appHubAppName""...","['Sandbox', 'Multiplayer', 'Funny', 'Moddable'...",ITS GOOD,True,True
3,76561199162775639,4000,"<div class=""apphub_AppName"" id=""appHubAppName""...","['Sandbox', 'Multiplayer', 'Funny', 'Moddable'...",fgvfbfcfdbfbfdbdfcv,True,True
4,76561198880207878,4000,"<div class=""apphub_AppName"" id=""appHubAppName""...","['Sandbox', 'Multiplayer', 'Funny', 'Moddable'...",gary,False,True
...,...,...,...,...,...,...,...
893715,76561198208834272,203290,"<div class=""apphub_AppName"" id=""appHubAppName""...","['Free to Play', 'FPS', 'Action', 'Multiplayer...",Bullets Actually Work!!! Hit detection is the ...,True,True
893716,76561198160362066,203290,"<div class=""apphub_AppName"" id=""appHubAppName""...","['Free to Play', 'FPS', 'Action', 'Multiplayer...",good.,True,True
893717,76561198314101022,203290,"<div class=""apphub_AppName"" id=""appHubAppName""...","['Free to Play', 'FPS', 'Action', 'Multiplayer...",xD Nice AMERICA\n,True,True
893718,76561198333531178,203290,"<div class=""apphub_AppName"" id=""appHubAppName""...","['Free to Play', 'FPS', 'Action', 'Multiplayer...",used to be a fun game and a laugh. now it just...,False,True


In [8]:
df_2 = pd.read_csv('../../../data/fps_trimmed_02.csv', error_bad_lines=False, engine="python")
df_2

Skipping line 10423: field larger than field limit (131072)
Skipping line 831271: field larger than field limit (131072)
Skipping line 1017850: field larger than field limit (131072)
Skipping line 1190827: field larger than field limit (131072)


,steamid,appid,app_title,app_tags,review,voted_up,fps
0,76561198107294407,10,"<div class=""apphub_AppName"" id=""appHubAppName""...","['Action', 'FPS', 'Multiplayer', 'Shooter', 'C...",best gameplay ever created,True,True
1,76561198011733201,10,"<div class=""apphub_AppName"" id=""appHubAppName""...","['Action', 'FPS', 'Multiplayer', 'Shooter', 'C...",Timeless. Thank you volvo.,True,True
2,76561198168961276,10,"<div class=""apphub_AppName"" id=""appHubAppName""...","['Action', 'FPS', 'Multiplayer', 'Shooter', 'C...",this game literally made me love guns and fps ...,True,True
3,76561198957877160,10,"<div class=""apphub_AppName"" id=""appHubAppName""...","['Action', 'FPS', 'Multiplayer', 'Shooter', 'C...",If this gets a rework in 2021 it's going to be...,True,True
4,76561199050314447,10,"<div class=""apphub_AppName"" id=""appHubAppName""...","['Action', 'FPS', 'Multiplayer', 'Shooter', 'C...",ok,True,True
...,...,...,...,...,...,...,...
1405359,76561198019403015,3730,"<div class=""apphub_AppName"" id=""appHubAppName""...","['FPS', 'Action', 'Classic', 'Aliens', 'Horror...",Superb multiplayer fun compared to AvP2 and AvP,True,True
1405360,76561198020998869,3730,"<div class=""apphub_AppName"" id=""appHubAppName""...","['FPS', 'Action', 'Classic', 'Aliens', 'Horror...",itÂ´s just a classic game for a cheap price ;),True,True
1405361,76561197970059401,3730,"<div class=""apphub_AppName"" id=""appHubAppName""...","['FPS', 'Action', 'Classic', 'Aliens', 'Horror...","It's pure dead brilliant,",True,True
1405362,76561197967992446,3730,"<div class=""apphub_AppName"" id=""appHubAppName""...","['FPS', 'Action', 'Classic', 'Aliens', 'Horror...",Here's a fun thing to do: Play Aliens Vs. Pred...,True,True


In [9]:
df_3 = pd.read_csv('../../../data/fps_trimmed_03.csv', error_bad_lines=False, engine="python")
df_3

Skipping line 10423: field larger than field limit (131072)
Skipping line 831271: field larger than field limit (131072)
Skipping line 1017850: field larger than field limit (131072)
Skipping line 1190827: field larger than field limit (131072)


,steamid,appid,app_title,app_tags,review,voted_up,fps
0,76561198107294407,10,"<div class=""apphub_AppName"" id=""appHubAppName""...","['Action', 'FPS', 'Multiplayer', 'Shooter', 'C...",best gameplay ever created,True,True
1,76561198011733201,10,"<div class=""apphub_AppName"" id=""appHubAppName""...","['Action', 'FPS', 'Multiplayer', 'Shooter', 'C...",Timeless. Thank you volvo.,True,True
2,76561198168961276,10,"<div class=""apphub_AppName"" id=""appHubAppName""...","['Action', 'FPS', 'Multiplayer', 'Shooter', 'C...",this game literally made me love guns and fps ...,True,True
3,76561198957877160,10,"<div class=""apphub_AppName"" id=""appHubAppName""...","['Action', 'FPS', 'Multiplayer', 'Shooter', 'C...",If this gets a rework in 2021 it's going to be...,True,True
4,76561199050314447,10,"<div class=""apphub_AppName"" id=""appHubAppName""...","['Action', 'FPS', 'Multiplayer', 'Shooter', 'C...",ok,True,True
...,...,...,...,...,...,...,...
1405359,76561198019403015,3730,"<div class=""apphub_AppName"" id=""appHubAppName""...","['FPS', 'Action', 'Classic', 'Aliens', 'Horror...",Superb multiplayer fun compared to AvP2 and AvP,True,True
1405360,76561198020998869,3730,"<div class=""apphub_AppName"" id=""appHubAppName""...","['FPS', 'Action', 'Classic', 'Aliens', 'Horror...",itÂ´s just a classic game for a cheap price ;),True,True
1405361,76561197970059401,3730,"<div class=""apphub_AppName"" id=""appHubAppName""...","['FPS', 'Action', 'Classic', 'Aliens', 'Horror...","It's pure dead brilliant,",True,True
1405362,76561197967992446,3730,"<div class=""apphub_AppName"" id=""appHubAppName""...","['FPS', 'Action', 'Classic', 'Aliens', 'Horror...",Here's a fun thing to do: Play Aliens Vs. Pred...,True,True


In [10]:
df_4 = pd.read_csv('../../../data/fps_trimmed_04.csv', error_bad_lines=False, engine="python")
df_4

Skipping line 10423: field larger than field limit (131072)
Skipping line 831271: field larger than field limit (131072)
Skipping line 1017850: field larger than field limit (131072)
Skipping line 1190827: field larger than field limit (131072)


,steamid,appid,app_title,app_tags,review,voted_up,fps
0,76561198107294407,10,"<div class=""apphub_AppName"" id=""appHubAppName""...","['Action', 'FPS', 'Multiplayer', 'Shooter', 'C...",best gameplay ever created,True,True
1,76561198011733201,10,"<div class=""apphub_AppName"" id=""appHubAppName""...","['Action', 'FPS', 'Multiplayer', 'Shooter', 'C...",Timeless. Thank you volvo.,True,True
2,76561198168961276,10,"<div class=""apphub_AppName"" id=""appHubAppName""...","['Action', 'FPS', 'Multiplayer', 'Shooter', 'C...",this game literally made me love guns and fps ...,True,True
3,76561198957877160,10,"<div class=""apphub_AppName"" id=""appHubAppName""...","['Action', 'FPS', 'Multiplayer', 'Shooter', 'C...",If this gets a rework in 2021 it's going to be...,True,True
4,76561199050314447,10,"<div class=""apphub_AppName"" id=""appHubAppName""...","['Action', 'FPS', 'Multiplayer', 'Shooter', 'C...",ok,True,True
...,...,...,...,...,...,...,...
1405359,76561198019403015,3730,"<div class=""apphub_AppName"" id=""appHubAppName""...","['FPS', 'Action', 'Classic', 'Aliens', 'Horror...",Superb multiplayer fun compared to AvP2 and AvP,True,True
1405360,76561198020998869,3730,"<div class=""apphub_AppName"" id=""appHubAppName""...","['FPS', 'Action', 'Classic', 'Aliens', 'Horror...",itÂ´s just a classic game for a cheap price ;),True,True
1405361,76561197970059401,3730,"<div class=""apphub_AppName"" id=""appHubAppName""...","['FPS', 'Action', 'Classic', 'Aliens', 'Horror...","It's pure dead brilliant,",True,True
1405362,76561197967992446,3730,"<div class=""apphub_AppName"" id=""appHubAppName""...","['FPS', 'Action', 'Classic', 'Aliens', 'Horror...",Here's a fun thing to do: Play Aliens Vs. Pred...,True,True


In [11]:
df_5 = pd.read_csv('../../../data/fps_trimmed_05.csv', error_bad_lines=False, engine="python")
df_5

Skipping line 10423: field larger than field limit (131072)
Skipping line 831271: field larger than field limit (131072)
Skipping line 1017850: field larger than field limit (131072)
Skipping line 1190827: field larger than field limit (131072)


,steamid,appid,app_title,app_tags,review,voted_up,fps
0,76561198107294407,10,"<div class=""apphub_AppName"" id=""appHubAppName""...","['Action', 'FPS', 'Multiplayer', 'Shooter', 'C...",best gameplay ever created,True,True
1,76561198011733201,10,"<div class=""apphub_AppName"" id=""appHubAppName""...","['Action', 'FPS', 'Multiplayer', 'Shooter', 'C...",Timeless. Thank you volvo.,True,True
2,76561198168961276,10,"<div class=""apphub_AppName"" id=""appHubAppName""...","['Action', 'FPS', 'Multiplayer', 'Shooter', 'C...",this game literally made me love guns and fps ...,True,True
3,76561198957877160,10,"<div class=""apphub_AppName"" id=""appHubAppName""...","['Action', 'FPS', 'Multiplayer', 'Shooter', 'C...",If this gets a rework in 2021 it's going to be...,True,True
4,76561199050314447,10,"<div class=""apphub_AppName"" id=""appHubAppName""...","['Action', 'FPS', 'Multiplayer', 'Shooter', 'C...",ok,True,True
...,...,...,...,...,...,...,...
1405359,76561198019403015,3730,"<div class=""apphub_AppName"" id=""appHubAppName""...","['FPS', 'Action', 'Classic', 'Aliens', 'Horror...",Superb multiplayer fun compared to AvP2 and AvP,True,True
1405360,76561198020998869,3730,"<div class=""apphub_AppName"" id=""appHubAppName""...","['FPS', 'Action', 'Classic', 'Aliens', 'Horror...",itÂ´s just a classic game for a cheap price ;),True,True
1405361,76561197970059401,3730,"<div class=""apphub_AppName"" id=""appHubAppName""...","['FPS', 'Action', 'Classic', 'Aliens', 'Horror...","It's pure dead brilliant,",True,True
1405362,76561197967992446,3730,"<div class=""apphub_AppName"" id=""appHubAppName""...","['FPS', 'Action', 'Classic', 'Aliens', 'Horror...",Here's a fun thing to do: Play Aliens Vs. Pred...,True,True


In [12]:
df_6 = pd.read_csv('../../../data/fps_trimmed_06.csv', error_bad_lines=False, engine="python")
df_6

Skipping line 10423: field larger than field limit (131072)
Skipping line 831271: field larger than field limit (131072)
Skipping line 1017850: field larger than field limit (131072)
Skipping line 1190827: field larger than field limit (131072)


,steamid,appid,app_title,app_tags,review,voted_up,fps
0,76561198107294407,10,"<div class=""apphub_AppName"" id=""appHubAppName""...","['Action', 'FPS', 'Multiplayer', 'Shooter', 'C...",best gameplay ever created,True,True
1,76561198011733201,10,"<div class=""apphub_AppName"" id=""appHubAppName""...","['Action', 'FPS', 'Multiplayer', 'Shooter', 'C...",Timeless. Thank you volvo.,True,True
2,76561198168961276,10,"<div class=""apphub_AppName"" id=""appHubAppName""...","['Action', 'FPS', 'Multiplayer', 'Shooter', 'C...",this game literally made me love guns and fps ...,True,True
3,76561198957877160,10,"<div class=""apphub_AppName"" id=""appHubAppName""...","['Action', 'FPS', 'Multiplayer', 'Shooter', 'C...",If this gets a rework in 2021 it's going to be...,True,True
4,76561199050314447,10,"<div class=""apphub_AppName"" id=""appHubAppName""...","['Action', 'FPS', 'Multiplayer', 'Shooter', 'C...",ok,True,True
...,...,...,...,...,...,...,...
1405359,76561198019403015,3730,"<div class=""apphub_AppName"" id=""appHubAppName""...","['FPS', 'Action', 'Classic', 'Aliens', 'Horror...",Superb multiplayer fun compared to AvP2 and AvP,True,True
1405360,76561198020998869,3730,"<div class=""apphub_AppName"" id=""appHubAppName""...","['FPS', 'Action', 'Classic', 'Aliens', 'Horror...",itÂ´s just a classic game for a cheap price ;),True,True
1405361,76561197970059401,3730,"<div class=""apphub_AppName"" id=""appHubAppName""...","['FPS', 'Action', 'Classic', 'Aliens', 'Horror...","It's pure dead brilliant,",True,True
1405362,76561197967992446,3730,"<div class=""apphub_AppName"" id=""appHubAppName""...","['FPS', 'Action', 'Classic', 'Aliens', 'Horror...",Here's a fun thing to do: Play Aliens Vs. Pred...,True,True


In [13]:
df_7 = pd.read_csv('../../../data/fps_trimmed_07.csv', error_bad_lines=False, engine="python")
df_7

Skipping line 10423: field larger than field limit (131072)
Skipping line 831271: field larger than field limit (131072)
Skipping line 1017850: field larger than field limit (131072)
Skipping line 1190827: field larger than field limit (131072)


,steamid,appid,app_title,app_tags,review,voted_up,fps
0,76561198107294407,10,"<div class=""apphub_AppName"" id=""appHubAppName""...","['Action', 'FPS', 'Multiplayer', 'Shooter', 'C...",best gameplay ever created,True,True
1,76561198011733201,10,"<div class=""apphub_AppName"" id=""appHubAppName""...","['Action', 'FPS', 'Multiplayer', 'Shooter', 'C...",Timeless. Thank you volvo.,True,True
2,76561198168961276,10,"<div class=""apphub_AppName"" id=""appHubAppName""...","['Action', 'FPS', 'Multiplayer', 'Shooter', 'C...",this game literally made me love guns and fps ...,True,True
3,76561198957877160,10,"<div class=""apphub_AppName"" id=""appHubAppName""...","['Action', 'FPS', 'Multiplayer', 'Shooter', 'C...",If this gets a rework in 2021 it's going to be...,True,True
4,76561199050314447,10,"<div class=""apphub_AppName"" id=""appHubAppName""...","['Action', 'FPS', 'Multiplayer', 'Shooter', 'C...",ok,True,True
...,...,...,...,...,...,...,...
1405359,76561198019403015,3730,"<div class=""apphub_AppName"" id=""appHubAppName""...","['FPS', 'Action', 'Classic', 'Aliens', 'Horror...",Superb multiplayer fun compared to AvP2 and AvP,True,True
1405360,76561198020998869,3730,"<div class=""apphub_AppName"" id=""appHubAppName""...","['FPS', 'Action', 'Classic', 'Aliens', 'Horror...",itÂ´s just a classic game for a cheap price ;),True,True
1405361,76561197970059401,3730,"<div class=""apphub_AppName"" id=""appHubAppName""...","['FPS', 'Action', 'Classic', 'Aliens', 'Horror...","It's pure dead brilliant,",True,True
1405362,76561197967992446,3730,"<div class=""apphub_AppName"" id=""appHubAppName""...","['FPS', 'Action', 'Classic', 'Aliens', 'Horror...",Here's a fun thing to do: Play Aliens Vs. Pred...,True,True


In [14]:
df_8 = pd.read_csv('../../../data/fps_trimmed_08.csv', error_bad_lines=False, engine="python")
df_8

Skipping line 10423: field larger than field limit (131072)
Skipping line 831271: field larger than field limit (131072)
Skipping line 1017850: field larger than field limit (131072)
Skipping line 1190827: field larger than field limit (131072)


,steamid,appid,app_title,app_tags,review,voted_up,fps
0,76561198107294407,10,"<div class=""apphub_AppName"" id=""appHubAppName""...","['Action', 'FPS', 'Multiplayer', 'Shooter', 'C...",best gameplay ever created,True,True
1,76561198011733201,10,"<div class=""apphub_AppName"" id=""appHubAppName""...","['Action', 'FPS', 'Multiplayer', 'Shooter', 'C...",Timeless. Thank you volvo.,True,True
2,76561198168961276,10,"<div class=""apphub_AppName"" id=""appHubAppName""...","['Action', 'FPS', 'Multiplayer', 'Shooter', 'C...",this game literally made me love guns and fps ...,True,True
3,76561198957877160,10,"<div class=""apphub_AppName"" id=""appHubAppName""...","['Action', 'FPS', 'Multiplayer', 'Shooter', 'C...",If this gets a rework in 2021 it's going to be...,True,True
4,76561199050314447,10,"<div class=""apphub_AppName"" id=""appHubAppName""...","['Action', 'FPS', 'Multiplayer', 'Shooter', 'C...",ok,True,True
...,...,...,...,...,...,...,...
1405359,76561198019403015,3730,"<div class=""apphub_AppName"" id=""appHubAppName""...","['FPS', 'Action', 'Classic', 'Aliens', 'Horror...",Superb multiplayer fun compared to AvP2 and AvP,True,True
1405360,76561198020998869,3730,"<div class=""apphub_AppName"" id=""appHubAppName""...","['FPS', 'Action', 'Classic', 'Aliens', 'Horror...",itÂ´s just a classic game for a cheap price ;),True,True
1405361,76561197970059401,3730,"<div class=""apphub_AppName"" id=""appHubAppName""...","['FPS', 'Action', 'Classic', 'Aliens', 'Horror...","It's pure dead brilliant,",True,True
1405362,76561197967992446,3730,"<div class=""apphub_AppName"" id=""appHubAppName""...","['FPS', 'Action', 'Classic', 'Aliens', 'Horror...",Here's a fun thing to do: Play Aliens Vs. Pred...,True,True


In [15]:
df_9 = pd.read_csv('../../../data/fps_trimmed_09.csv', error_bad_lines=False, engine="python")
df_9

Skipping line 10423: field larger than field limit (131072)
Skipping line 831271: field larger than field limit (131072)
Skipping line 1017850: field larger than field limit (131072)
Skipping line 1190827: field larger than field limit (131072)


,steamid,appid,app_title,app_tags,review,voted_up,fps
0,76561198107294407,10,"<div class=""apphub_AppName"" id=""appHubAppName""...","['Action', 'FPS', 'Multiplayer', 'Shooter', 'C...",best gameplay ever created,True,True
1,76561198011733201,10,"<div class=""apphub_AppName"" id=""appHubAppName""...","['Action', 'FPS', 'Multiplayer', 'Shooter', 'C...",Timeless. Thank you volvo.,True,True
2,76561198168961276,10,"<div class=""apphub_AppName"" id=""appHubAppName""...","['Action', 'FPS', 'Multiplayer', 'Shooter', 'C...",this game literally made me love guns and fps ...,True,True
3,76561198957877160,10,"<div class=""apphub_AppName"" id=""appHubAppName""...","['Action', 'FPS', 'Multiplayer', 'Shooter', 'C...",If this gets a rework in 2021 it's going to be...,True,True
4,76561199050314447,10,"<div class=""apphub_AppName"" id=""appHubAppName""...","['Action', 'FPS', 'Multiplayer', 'Shooter', 'C...",ok,True,True
...,...,...,...,...,...,...,...
1405359,76561198019403015,3730,"<div class=""apphub_AppName"" id=""appHubAppName""...","['FPS', 'Action', 'Classic', 'Aliens', 'Horror...",Superb multiplayer fun compared to AvP2 and AvP,True,True
1405360,76561198020998869,3730,"<div class=""apphub_AppName"" id=""appHubAppName""...","['FPS', 'Action', 'Classic', 'Aliens', 'Horror...",itÂ´s just a classic game for a cheap price ;),True,True
1405361,76561197970059401,3730,"<div class=""apphub_AppName"" id=""appHubAppName""...","['FPS', 'Action', 'Classic', 'Aliens', 'Horror...","It's pure dead brilliant,",True,True
1405362,76561197967992446,3730,"<div class=""apphub_AppName"" id=""appHubAppName""...","['FPS', 'Action', 'Classic', 'Aliens', 'Horror...",Here's a fun thing to do: Play Aliens Vs. Pred...,True,True


In [16]:
df_10 = pd.read_csv('../../../data/fps_trimmed_10.csv', error_bad_lines=False, engine="python")
df_10

Skipping line 10423: field larger than field limit (131072)
Skipping line 831271: field larger than field limit (131072)
Skipping line 1017850: field larger than field limit (131072)
Skipping line 1190827: field larger than field limit (131072)


,steamid,appid,app_title,app_tags,review,voted_up,fps
0,76561198107294407,10,"<div class=""apphub_AppName"" id=""appHubAppName""...","['Action', 'FPS', 'Multiplayer', 'Shooter', 'C...",best gameplay ever created,True,True
1,76561198011733201,10,"<div class=""apphub_AppName"" id=""appHubAppName""...","['Action', 'FPS', 'Multiplayer', 'Shooter', 'C...",Timeless. Thank you volvo.,True,True
2,76561198168961276,10,"<div class=""apphub_AppName"" id=""appHubAppName""...","['Action', 'FPS', 'Multiplayer', 'Shooter', 'C...",this game literally made me love guns and fps ...,True,True
3,76561198957877160,10,"<div class=""apphub_AppName"" id=""appHubAppName""...","['Action', 'FPS', 'Multiplayer', 'Shooter', 'C...",If this gets a rework in 2021 it's going to be...,True,True
4,76561199050314447,10,"<div class=""apphub_AppName"" id=""appHubAppName""...","['Action', 'FPS', 'Multiplayer', 'Shooter', 'C...",ok,True,True
...,...,...,...,...,...,...,...
1405359,76561198019403015,3730,"<div class=""apphub_AppName"" id=""appHubAppName""...","['FPS', 'Action', 'Classic', 'Aliens', 'Horror...",Superb multiplayer fun compared to AvP2 and AvP,True,True
1405360,76561198020998869,3730,"<div class=""apphub_AppName"" id=""appHubAppName""...","['FPS', 'Action', 'Classic', 'Aliens', 'Horror...",itÂ´s just a classic game for a cheap price ;),True,True
1405361,76561197970059401,3730,"<div class=""apphub_AppName"" id=""appHubAppName""...","['FPS', 'Action', 'Classic', 'Aliens', 'Horror...","It's pure dead brilliant,",True,True
1405362,76561197967992446,3730,"<div class=""apphub_AppName"" id=""appHubAppName""...","['FPS', 'Action', 'Classic', 'Aliens', 'Horror...",Here's a fun thing to do: Play Aliens Vs. Pred...,True,True


In [20]:
all_data_fps = pd.concat([df_0, df_1, df_2, df_3, df_4, df_5, df_6, df_7, df_8, df_9, df_10])

In [30]:
all_data_fps.dropna()

,steamid,appid,app_title,app_tags,review,voted_up,fps
0,76561198107294407,10,"<div class=""apphub_AppName"" id=""appHubAppName""...","['Action', 'FPS', 'Multiplayer', 'Shooter', 'C...",best gameplay ever created,True,True
1,76561198011733201,10,"<div class=""apphub_AppName"" id=""appHubAppName""...","['Action', 'FPS', 'Multiplayer', 'Shooter', 'C...",Timeless. Thank you volvo.,True,True
2,76561198168961276,10,"<div class=""apphub_AppName"" id=""appHubAppName""...","['Action', 'FPS', 'Multiplayer', 'Shooter', 'C...",this game literally made me love guns and fps ...,True,True
3,76561198957877160,10,"<div class=""apphub_AppName"" id=""appHubAppName""...","['Action', 'FPS', 'Multiplayer', 'Shooter', 'C...",If this gets a rework in 2021 it's going to be...,True,True
4,76561199050314447,10,"<div class=""apphub_AppName"" id=""appHubAppName""...","['Action', 'FPS', 'Multiplayer', 'Shooter', 'C...",ok,True,True
...,...,...,...,...,...,...,...
1405359,76561198019403015,3730,"<div class=""apphub_AppName"" id=""appHubAppName""...","['FPS', 'Action', 'Classic', 'Aliens', 'Horror...",Superb multiplayer fun compared to AvP2 and AvP,True,True
1405360,76561198020998869,3730,"<div class=""apphub_AppName"" id=""appHubAppName""...","['FPS', 'Action', 'Classic', 'Aliens', 'Horror...",itÂ´s just a classic game for a cheap price ;),True,True
1405361,76561197970059401,3730,"<div class=""apphub_AppName"" id=""appHubAppName""...","['FPS', 'Action', 'Classic', 'Aliens', 'Horror...","It's pure dead brilliant,",True,True
1405362,76561197967992446,3730,"<div class=""apphub_AppName"" id=""appHubAppName""...","['FPS', 'Action', 'Classic', 'Aliens', 'Horror...",Here's a fun thing to do: Play Aliens Vs. Pred...,True,True


In [31]:
adf = all_data_fps.drop(['review', 'app_tags', 'fps', 'appid'])
adf.head()

TypeError: ufunc 'isnan' not supported for the input types, and the inputs could not be safely coerced to any supported types according to the casting rule ''safe''